###### <h2> Exercise 9 - Autocorrelation</h2>

We now consider a linear specification $y_t=\alpha + \beta_1x_{1,t} + \beta_2x_{2,t} +\epsilon_t$ in which the error term $\epsilon_t$ is now correlated with previous values,
\begin{align}
\epsilon_t = \rho\epsilon_{t-1} + u_t,\ \ \ u_t\sim N(0,1)
\end{align}
We begin by performing several $t$ tests when we set $\beta_1=0$.

In [47]:
using Distributions
using PyPlot
using LinearAlgebra
using Printf
using Random

TypeII = 0;

d = Normal(0,1);
N = 500;

alpha = 0.78;
beta1 = 0.5;
beta2 = 5.61;

rho = 0.99;

M = 1000;
betas = zeros(M);

for ii=1:M

    x1 = rand(d,N);
    x2 = rand(d,N);

    epsInit = 0.0
    for ii=1:1000
        epsInit = rho*epsInit + rand(Normal(0.0,1.0))
    end
    eps = zeros(N);
    eps[1] = epsInit;
    for ii=2:N
        eps[ii] = rho*eps[ii-1] + rand(Normal(0.0,1.0))
    end

    y = alpha .+ beta1*x1 + beta2*x2 + eps;

    X = [ones(N) x1 x2];

    betaHat = (X'*X)\(X'*y);
    epsHat = y - X*betaHat
    SigHat = inv(X'*X)*(epsHat'*epsHat)
    
    betaHat1 = betaHat[2]
    betas[ii] = betaHat1
    stdErr1 = sqrt(SigHat[2,2])/sqrt(N-3)
    tHat1 = betaHat1/stdErr1;
    p1 = cdf(TDist(3),-abs(tHat1)) + (1-cdf(TDist(3),abs(tHat1)))
    
    if(p1>0.05)
        TypeII = TypeII + 1;
    end
    
end

println("Average coefficient estimate: ", mean(betas))
println("Number of Type II errors: ", TypeII)

Average coefficient estimate: 0.5079074264405374
Number of Type II errors: 809


As with heteroskedasticity, we obtain reasonable parameter estimates, but a huge proportion of Type II errors.  To check that this issue arises from autocorrelation, we repeat with $N(0,1)$ errors.

In [49]:
TypeII = 0

for ii=1:M

    x1 = rand(d,N);
    x2 = rand(d,N);

    eps = rand(Normal(),N);

    y = alpha .+ beta1*x1 + beta2*x2 + eps;

    X = [ones(N) x1 x2];

    betaHat = (X'*X)\(X'*y);
    epsHat = y - X*betaHat
    SigHat = inv(X'*X)*(epsHat'*epsHat)
    
    betaHat1 = betaHat[2]
    betas[ii] = betaHat1
    stdErr1 = sqrt(SigHat[2,2])/sqrt(N-3)
    tHat1 = betaHat1/stdErr1;
    p1 = cdf(TDist(3),-abs(tHat1)) + (1-cdf(TDist(3),abs(tHat1)))
    
    if(p1>0.05)
        TypeII = TypeII + 1;
    end
    
end

println("Average coefficient estimate: ", mean(betas))
println("Number of Type II errors: ", TypeII)

Average coefficient estimate: 0.5010950494860122
Number of Type II errors: 0


Removing the autocorrelation reduces the likelihood of a Type II error to almost nothing.  Moreover, reducing the persistence of the autocorrelation modestly disproportionately reduces the fraction of Type II errors reported.

In [56]:
TypeII = 0

rho = 0.98;

for ii=1:M

    x1 = rand(d,N);
    x2 = rand(d,N);

    epsInit = 0.0
    for ii=1:1000
        epsInit = rho*epsInit + rand(Normal(0.0,1.0))
    end
    eps = zeros(N);
    eps[1] = epsInit;
    for ii=2:N
        eps[ii] = rho*eps[ii-1] + rand(Normal(0.0,1.0))
    end

    y = alpha .+ beta1*x1 + beta2*x2 + eps;

    X = [ones(N) x1 x2];

    betaHat = (X'*X)\(X'*y);
    epsHat = y - X*betaHat
    SigHat = inv(X'*X)*(epsHat'*epsHat)
    
    betaHat1 = betaHat[2]
    betas[ii] = betaHat1
    stdErr1 = sqrt(SigHat[2,2])/sqrt(N-3)
    tHat1 = betaHat1/stdErr1;
    p1 = cdf(TDist(3),-abs(tHat1)) + (1-cdf(TDist(3),abs(tHat1)))
    
    if(p1>0.05)
        TypeII = TypeII + 1;
    end
    
end

println("Average coefficient estimate: ", mean(betas))
println("Number of Type II errors: ", TypeII)

Average coefficient estimate: 0.5032504668885712
Number of Type II errors: 712


With a persistence of 0.5 the errors essentially disappear entirely.